In [1]:
# !pip install --upgrade pip
# !pip install tensorflow-gpu=="2.*"
# !pip install numpy pandas

## Load Libraries & Datasets

In [2]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from scipy.io import savemat

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from keras.utils.np_utils import to_categorical

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, accuracy_score, matthews_corrcoef, balanced_accuracy_score, precision_recall_fscore_support
from sklearn.metrics import auc, average_precision_score, precision_recall_curve, roc_curve

2022-02-15 12:28:37.070614: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.1/lib64
2022-02-15 12:28:37.070632: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
from keras.models import Model
from keras.layers import Input, Dense, BatchNormalization, Dropout, Conv1D, Conv2D, MaxPooling2D, MaxPooling1D, Flatten
from keras import optimizers
from keras import metrics

from keras import backend as K
from keras.models import load_model
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from random import sample

import pdb

In [4]:
def prepare_dset(dset):
    dset = pd.concat([pd.DataFrame(np.asarray(dset.conc > 0.01, dtype='int')).rename(columns={0:'label'}), 
           dset.iloc[:, 3:]], axis=1)
    return dset

In [5]:
## raw data
pbno3_b1_raw = pd.read_csv("data/pbno3_batch1.csv")
pbno3_b2_raw = pd.read_csv("data/pbno3_batch2.csv")
pbno3_b1_raw = prepare_dset(pbno3_b1_raw)
pbno3_b2_raw = prepare_dset(pbno3_b2_raw)

# ## Power Spectrum Normalized
# pbno3_b1_psn = pd.read_csv("./data_final/pbno3_dset/new/pbno3_psn_batch1.csv")
# pbno3_b2_psn = pd.read_csv("./data_final/pbno3_dset/new/pbno3_psn_batch2.csv")
# pbno3_b1_psn = prepare_dset(pbno3_b1_psn)
# pbno3_b2_psn = prepare_dset(pbno3_b2_psn)

# ## Baseline Corrected
# pbno3_b1_bc = pd.read_csv("./data_final/pbno3_dset/new/pbno3_bl_batch1.csv")
# pbno3_b2_bc = pd.read_csv("./data_final/pbno3_dset/new/pbno3_bl_batch2.csv")
# pbno3_b1_bc = prepare_dset(pbno3_b1_bc)
# pbno3_b2_bc = prepare_dset(pbno3_b2_bc)

# ## Batch Normalized
# pbno3_b1_bn = pd.read_csv("./data_final/pbno3_dset/new/pbno3_bn_batch1.csv")
# pbno3_b2_bn = pd.read_csv("./data_final/pbno3_dset/new/pbno3_bn_batch2.csv")
# pbno3_b1_bn = prepare_dset(pbno3_b1_bn)
# pbno3_b2_bn = prepare_dset(pbno3_b2_bn)

# ## Baseline Correctd & Power Spectrum Normalized
# pbno3_b1_bl_psn = pd.read_csv("./data_final/pbno3_dset/new/pbno3_bl_psn_batch1.csv")
# pbno3_b2_bl_psn = pd.read_csv("./data_final/pbno3_dset/new/pbno3_bl_psn_batch2.csv")
# pbno3_b1_bl_psn = prepare_dset(pbno3_b1_bl_psn)
# pbno3_b2_bl_psn = prepare_dset(pbno3_b2_bl_psn)

# ## Baseline Correctd & Batch Normalized
# pbno3_b1_bl_bn = pd.read_csv("./data_final/pbno3_dset/new/pbno3_bl_bn_batch1.csv")
# pbno3_b2_bl_bn = pd.read_csv("./data_final/pbno3_dset/new/pbno3_bl_bn_batch2.csv")
# pbno3_b1_bl_bn = prepare_dset(pbno3_b1_bl_bn)
# pbno3_b2_bl_bn = prepare_dset(pbno3_b2_bl_bn)

## Define Util Functions

In [6]:
## Define performance measures
def yoden_index(y, y_pred):
    tn, fp, fn, tp = confusion_matrix(y, y_pred, labels=[0,1]).ravel()
    j = (tp/(tp+fn)) + (tn/(tn+fp)) - 1
    return j

def pmeasure(y, y_pred):
    tn, fp, fn, tp = confusion_matrix(y, y_pred, labels=[0,1]).ravel()
    sensitivity = tp / (tp + fn )
    specificity = tn / (tn + fp)
    f1score = (2 * tp) / (2 * tp + fp + fn)
    return ({'Sensitivity': sensitivity, 'Specificity': specificity, 'F1-Score': f1score})

def Show_Statistics(msg,mean_Stats, sd_Stats, sigfig):
    print(msg.upper())
    print(70*'-')
    print('Accuracy:{} + {}'          .format(round(mean_Stats[0],sigfig), round(sd_Stats[0],sigfig)))
    print('Sensitivity:{} + {} '      .format(round(mean_Stats[1],sigfig), round(sd_Stats[1],sigfig)))
    print('Specificity:{} + {}'       .format(round(mean_Stats[2],sigfig), round(sd_Stats[2],sigfig)))
    print('F1-Score:{} + {}'          .format(round(mean_Stats[3],sigfig), round(sd_Stats[3],sigfig)))
    print('MCC:{} + {}'               .format(round(mean_Stats[4],sigfig), round(sd_Stats[4],sigfig)))
    print('Balance Accuracy:{} + {}'  .format(round(mean_Stats[5],sigfig), round(sd_Stats[5],sigfig)))
    print('Youden-Index:{} + {}'      .format(round(mean_Stats[6],sigfig), round(sd_Stats[6],sigfig)))
    print('AUC:{} + {}'               .format(round(mean_Stats[7],sigfig), round(sd_Stats[7],sigfig)))
    print('AUPR:{} + {}'              .format(round(mean_Stats[8],sigfig), round(sd_Stats[8],sigfig)))  
    print(70*'-')

def Calculate_Stats(y_actual,y_pred, y_score):
    acc = accuracy_score(y_actual, y_pred)
    sen = pmeasure(y_actual, y_pred)['Sensitivity']
    spe = pmeasure(y_actual, y_pred)['Specificity']
    f1 = pmeasure(y_actual, y_pred)['F1-Score']
    mcc = matthews_corrcoef(y_actual, y_pred)
    bacc = balanced_accuracy_score(y_actual, y_pred)
    yi = yoden_index(y_actual, y_pred)
    #auc = roc_auc_score(y_actual.argmax(axis=1), y_pred.argmax(axis=1))
  
    pre, rec, _ = precision_recall_curve(y_actual, y_score, pos_label=1)
    fpr, tpr, _ = roc_curve(y_actual, y_score, pos_label=1)
    auroc = auc(fpr, tpr)
    aupr = auc(rec, pre)
    
    return acc, sen, spe, f1, mcc, bacc, yi, auroc, aupr  

def label_by_th(y_pred, threshold=0.5):
    y_pred_copy = y_pred.copy()
    y_pred_copy[y_pred>= threshold] = 1 
    y_pred_copy[y_pred<threshold] = 0 
    return y_pred_copy

def cutoff_youdens_j(fpr,tpr,thresholds):
    j_scores = tpr-fpr
    j_ordered = sorted(zip(j_scores,thresholds))
    return j_ordered[-1][1]

In [7]:
def get_perf_summary(dataset, model, type):
    ## Prepare dataset
    dataset.loc[dataset.label>=1,"label"]=1
    
    ## Datasets 
    [X, y_true] = np.array(dataset.iloc[:,1:]), np.array(dataset.iloc[:,0])
    y_true = to_categorical(y_true)
    if type=="regression":
        y_score = model.predict(X, batch_size=1800, verbose=0)    
    else:
        y_score = model.predict(X, batch_size=1800, verbose=0)[:,1]
    #y_score = model.predict(X,batch_size=1800, verbose=0)
    #y_pred = to_categorical(get_label(y_score))      

    # Optimal Threshold
    fpr, tpr, thresholds_AUC = roc_curve(y_true.argmax(axis=1), y_score)
    precision, recall, thresholds_AUPR = precision_recall_curve(y_true.argmax(axis=1),y_score)

    ## Optimal Threshold metrics
    distance = (1-fpr)**2+(1-tpr)**2
    EERs = (1-recall)/(1-precision)
    positive = sum(y_true.argmax(axis=1))
    negative = y_true.shape[0]-positive
    ratio = negative/positive
    opt_t_AUC = thresholds_AUC[np.argmin(distance)]
    opt_t_AUPR = thresholds_AUPR[np.argmin(np.abs(EERs-ratio))]
    opt_yodens_j = cutoff_youdens_j(fpr, tpr, thresholds_AUC)        
    y_pred_th = label_by_th(y_score, opt_yodens_j)
    y_pred = to_categorical(y_pred_th)

    print(confusion_matrix(y_true.argmax(axis=1), y_pred.argmax(axis=1), labels=[0,1]).ravel())    

    ## Measures
    t_acc, t_sen, t_spe, t_f1, t_mcc, t_bacc, t_yi, t_auc, t_aupr = Calculate_Stats(y_true, y_pred, y_score)

    print("ACC: {}, Sen: {}, Spe: {}, F1: {}, MCC: {}, BACC: {}, YI: {}, AUC: {}, AUPR: {}".format(t_acc, t_sen, t_spe, t_f1, t_mcc, t_bacc, t_yi, t_auc, t_aupr))

In [8]:
def get_scale_dset(dset):
    res =[]
    tmp_dset = dset.iloc[:,1:]
    for i, row in tmp_dset.iterrows():
        res.append(row/max(abs(row)))
    
    res = pd.DataFrame(res).reset_index(drop=True)
    res = pd.concat([dset.iloc[:,0], res], axis=1)
    
    return res 

## Define Classifiers

In [9]:
def get_clf_res_ML(dev_dset, ind_test_dset, method):
    num_Trials = 10    
    dev_dset = get_scale_dset(dev_dset)
    ind_test_dset = get_scale_dset(ind_test_dset)

    # import warnings filter
    from warnings import simplefilter
    # ignore all future warnings
    simplefilter(action='ignore', category=FutureWarning)
    [DataX, LabelY] = np.array(dev_dset.iloc[:,1:]), np.array(dev_dset.iloc[:,0])    

    Stats =[]
    Stats_Ind =[]
    pdb.set_trace()
    for j in range(0, num_Trials):        
        X_train, X_val, y_train, y_val =\
        train_test_split(DataX, LabelY, test_size=0.2, stratify = LabelY)
        X_val, X_test, y_val, y_test =\
        train_test_split(X_val, y_val, test_size=0.5, stratify = y_val)

        if method == "LR":
            clf = LogisticRegression(random_state=0).fit(X_train, y_train)

        elif method == "LinSVM":
            clf = SVC(kernel = 'linear', probability=True).fit(X_train, y_train)

        elif method == "NB":
            clf = GaussianNB().fit(X_train, y_train)

        elif method == "DT":
            clf = DecisionTreeClassifier().fit(X_train, y_train)

        elif method == "MLP":
            clf = MLPClassifier().fit(X_train, y_train)

        elif method == "RF":
            clf = RandomForestClassifier().fit(X_train, y_train)

        elif method == "RBFSVM":
            clf = SVC(kernel = 'rbf', probability=True).fit(X_train, y_train)

        else:
            print("Please check avilable models")
            break;

        y_train_pred = clf.predict(X_train)
        y_train_score = clf.predict_proba(X_train)     
        y_train_pred = to_categorical(y_train_score.argmax(axis=1))
        y_train_score = y_train_score[:,1]

        y_val_pred = clf.predict(X_val)
        y_val_score = clf.predict_proba(X_val)     
        y_val_pred = to_categorical(y_val_score.argmax(axis=1))
        y_val_score = y_val_score[:,1]

        y_test_pred = clf.predict(X_test)
        y_test_score = clf.predict_proba(X_test) 
        y_test_pred = to_categorical(y_test_score.argmax(axis=1))
        y_test_score = y_test_score[:,1] 

        print(confusion_matrix(y_test, y_test_pred.argmax(axis=1), labels=[0,1]).ravel())

        y_train_pred = y_train_pred.argmax(axis=1)
        y_val_pred = y_val_pred.argmax(axis=1)
        y_test_pred = y_test_pred.argmax(axis=1)

        ## Training Measures
        tr_acc, tr_sen, tr_spe, tr_f1, tr_mcc, tr_bacc, tr_yi, tr_auc, tr_aupr = Calculate_Stats(y_train,y_train_pred, y_train_score);

        ## Validation Measures
        v_acc, v_sen, v_spe, v_f1, v_mcc, v_bacc, v_yi, v_auc, v_aupr = Calculate_Stats(y_val,y_val_pred, y_val_score);

        ## Test Measures
        t_acc, t_sen, t_spe, t_f1, t_mcc, t_bacc, t_yi, t_auc, t_aupr = Calculate_Stats(y_test,y_test_pred, y_test_score);

        Stats.append([tr_acc, tr_sen, tr_spe, tr_f1, tr_mcc, tr_bacc, tr_yi, tr_auc, tr_aupr,
                      v_acc, v_sen, v_spe, v_f1, v_mcc, v_bacc, v_yi, v_auc, v_aupr,
                      t_acc, t_sen, t_spe, t_f1, t_mcc, t_bacc, t_yi, t_auc, t_aupr])  

        print(' \nTraining/ Validation / Test BACC :', tr_bacc,'/',v_bacc,'/',t_bacc,
              ' \nTraining/ Validation / Test Youden-index:', tr_yi,'/',v_yi,'/',t_yi,
              ' \nTraining/ Validation / Test MCC:', tr_mcc,'/',v_mcc,'/',t_mcc,
              ' \nTraining/ Validation / Test AUC:', tr_auc,'/',v_auc,'/',t_auc,
              ' \nTraining/ Validation / Test AUPR:', tr_aupr,'/',v_aupr,'/',t_aupr)    

        ## Independent Test
        [Xt1, Yt1] = np.array(ind_test_dset.iloc[:,1:]), np.array(ind_test_dset.iloc[:,0])

        Yt1_pred = clf.predict(Xt1)
        Yt1_score = clf.predict_proba(Xt1)

        Stats_Ind.append(Calculate_Stats(Yt1, Yt1_pred, Yt1_score[:,1]))

        print(Stats_Ind)
    
    return Stats, Stats_Ind

In [10]:
def format_perf_mat(perf_list, model_type):
    perf_tbl = pd.DataFrame(perf_list)
    perf_tbl = pd.concat([perf_tbl, pd.DataFrame(np.repeat(model_type, len(perf_list)))], axis=1)
    perf_tbl.columns = ['Accuracy', "Sensitivity", "Specificity", "F1", "MCC", "BACC", 
                        "Yoden Index", "AUROC", "AUPR", "Model"]    
    return perf_tbl

In [11]:
def get_all_clf_res(devset, indset, species, prep_method):
    # NB_cv, NB_ind = get_clf_res_ML(devset, indset, method='NB')
    # DT_cv, DT_ind = get_clf_res_ML(devset, indset, method='DT')
    # LR_cv, LR_ind = get_clf_res_ML(devset, indset, method='LR')
    # RF_cv, RF_ind = get_clf_res_ML(devset, indset, method='RF')
    # LinSVM_cv, LinSVM_ind = get_clf_res_ML(devset, indset, method='LinSVM')
    # RBFSVM_cv, RBFSVM_ind = get_clf_res_ML(devset, indset, method='RBFSVM')
    MLP_cv, MLP_ind = get_clf_res_ML(devset, indset, method='MLP')
    
    fold_ind_perf = pd.concat([
                            # format_perf_mat(NB_ind, 'NB'), 
                            # format_perf_mat(DT_ind, 'DT'), 
                            # format_perf_mat(LR_ind, 'LR'),
                            # format_perf_mat(RF_ind, 'RF'), 
                            # format_perf_mat(LinSVM_ind, 'LinSVM'),
                            # format_perf_mat(RBFSVM_ind, 'RBFSVM'),
                            format_perf_mat(MLP_ind, 'MLP')
                               ], axis=0)

    return fold_ind_perf

## Train and Evaluate Performance of Models

In [12]:
# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

# perf_raw_pbno3_1 = get_all_clf_res(pbno3_b1_raw, pbno3_b2_raw, 'pbno3_b1b2', 'raw')
#perf_raw_pbno3_2 = get_all_clf_res(pbno3_b2_raw, pbno3_b1_raw, 'pbno3_b2b1', 'raw')

#
# var1, var2 = get_clf_res_ML(pbno3_b1_raw, pbno3_b2_raw, method='MLP')

######################################################################
print('Training MLP on batch 1...')
dset = get_scale_dset(pbno3_b1_raw)
[X_src, Y_src] = np.array(dset.iloc[:,1:]), np.array(dset.iloc[:,0])
#
clf = MLPClassifier().fit(X_src, Y_src)
#
y_test_pred = clf.predict(X_src)
y_test_score = clf.predict_proba(X_src) 
y_test_pred = to_categorical(y_test_score.argmax(axis=1))
y_test_pred = y_test_pred.argmax(axis=1)
y_test_score = y_test_score[:,1] 
#
print('CM: ', confusion_matrix(Y_src, y_test_pred, labels=[0,1]).ravel())
## Test Measures
t_acc, t_sen, t_spe, t_f1, t_mcc, t_bacc, t_yi, t_auc, t_aupr = Calculate_Stats(Y_src,y_test_pred, y_test_score)
print('t_acc, t_sen, t_spe, t_f1, t_mcc, t_bacc, t_yi, t_auc, t_aupr: ', t_acc, t_sen, t_spe, t_f1, t_mcc, t_bacc, t_yi, t_auc, t_aupr )

######################################################################
print('Testing trained MLP on batch 2...')
dset = get_scale_dset(pbno3_b2_raw)
[X_src, Y_src] = np.array(dset.iloc[:,1:]), np.array(dset.iloc[:,0])
#
# clf = MLPClassifier().fit(X_src, Y_src)
#
y_test_pred = clf.predict(X_src)
y_test_score = clf.predict_proba(X_src) 
y_test_pred = to_categorical(y_test_score.argmax(axis=1))
y_test_pred = y_test_pred.argmax(axis=1)
y_test_score = y_test_score[:,1] 
#
print('CM: ', confusion_matrix(Y_src, y_test_pred, labels=[0,1]).ravel())
## Test Measures
t_acc, t_sen, t_spe, t_f1, t_mcc, t_bacc, t_yi, t_auc, t_aupr = Calculate_Stats(Y_src,y_test_pred, y_test_score)
print('t_acc, t_sen, t_spe, t_f1, t_mcc, t_bacc, t_yi, t_auc, t_aupr: ', t_acc, t_sen, t_spe, t_f1, t_mcc, t_bacc, t_yi, t_auc, t_aupr )


Training MLP on batch 1...
CM:  [ 500    0    0 1500]
t_acc, t_sen, t_spe, t_f1, t_mcc, t_bacc, t_yi, t_auc, t_aupr:  1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0
Testing trained MLP on batch 2...
CM:  [494   6 991 509]
t_acc, t_sen, t_spe, t_f1, t_mcc, t_bacc, t_yi, t_auc, t_aupr:  0.5015 0.3393333333333333 0.988 0.5052109181141439 0.32415596923550277 0.6636666666666666 0.32733333333333325 0.6034333333333333 0.871441820700748


In [ ]:
perf_raw_pbno3_1.groupby('Model').mean()

In [ ]:
perf_raw_pbno3_2.groupby('Model').mean()

In [ ]:
pd.concat([perf_raw_pbno3_1, perf_raw_pbno3_2], axis=0).groupby('Model').mean()

In [ ]:
# perf_psn_pbno3_1 = get_all_clf_res(pbno3_b1_psn, pbno3_b2_psn, 'pbno3_b1b2', 'psn')
# perf_psn_pbno3_2 = get_all_clf_res(pbno3_b2_psn, pbno3_b1_psn, 'pbno3_b2b1', 'psn')

In [ ]:
# perf_psn_pbno3_1.groupby('Model').mean()

In [ ]:
# perf_psn_pbno3_2.groupby('Model').mean()

In [ ]:
# pd.concat([perf_psn_pbno3_1, perf_psn_pbno3_2], axis=0).groupby('Model').mean()

In [ ]:
# perf_bc_pbno3_1 = get_all_clf_res(pbno3_b1_bc, pbno3_b2_bc, 'pbno3_b1b2', 'bc')
# perf_bc_pbno3_2 = get_all_clf_res(pbno3_b2_bc, pbno3_b1_bc, 'pbno3_b2b1', 'bc')

In [ ]:
# perf_bc_pbno3_1.groupby('Model').mean()

In [ ]:
# perf_bc_pbno3_2.groupby('Model').mean()

In [ ]:
# pd.concat([perf_bc_pbno3_1, perf_bc_pbno3_2], axis=0).groupby('Model').mean()

In [ ]:
# perf_bn_pbno3_1 = get_all_clf_res(pbno3_b1_bn, pbno3_b2_bn, 'pbno3_b1b2', 'bn')
# perf_bn_pbno3_2 = get_all_clf_res(pbno3_b2_bn, pbno3_b1_bn, 'pbno3_b2b1', 'bn')

In [ ]:
# perf_bn_pbno3_1.groupby('Model').mean()

In [ ]:
# perf_bn_pbno3_2.groupby('Model').mean()

In [ ]:
# pd.concat([perf_bn_pbno3_1, perf_bn_pbno3_2], axis=0).groupby('Model').mean()

In [ ]:
# perf_bc_psn_pbno3_1 = get_all_clf_res(pbno3_b1_bl_psn, pbno3_b2_bl_psn, 'pbno3_b1b2', 'bc_psn')
# perf_bc_psn_pbno3_2 = get_all_clf_res(pbno3_b2_bl_psn, pbno3_b1_bl_psn, 'pbno3_b2b1', 'bc_psn')

In [ ]:
# perf_bc_psn_pbno3_1.groupby('Model').mean()

In [ ]:
# perf_bc_psn_pbno3_2.groupby('Model').mean()

In [ ]:
# pd.concat([perf_bc_psn_pbno3_1, perf_bc_psn_pbno3_2], axis=0).groupby('Model').mean()

In [ ]:
# perf_bc_bn_pbno3_1 = get_all_clf_res(pbno3_b1_bl_bn, pbno3_b2_bl_bn, 'pbno3_b1b2', 'bc_bn')
# perf_bc_bn_pbno3_2 = get_all_clf_res(pbno3_b2_bl_bn, pbno3_b1_bl_bn, 'pbno3_b2b1', 'bc_bn')

In [ ]:
# perf_bc_bn_pbno3_1.groupby('Model').mean()

In [ ]:
# perf_bc_bn_pbno3_2.groupby('Model').mean()

In [ ]:
# pd.concat([perf_bc_bn_pbno3_1, perf_bc_bn_pbno3_2], axis=0).groupby('Model').mean()